## Mínimos Cuadrados Generalizados (GLS)

El método de **Mínimos Cuadrados Generalizados** (*Generalized Least Squares*, GLS) es una extensión del modelo clásico de regresión por **mínimos cuadrados ordinarios (OLS)**. Se utiliza cuando no se cumplen los supuestos fundamentales de OLS, en particular:

- La **homocedasticidad**: varianza constante de los errores,
- La **no correlación entre errores**: independencia entre observaciones.

Cuando estos supuestos se violan, las estimaciones OLS siguen siendo **insesgadas**, pero **ya no son eficientes** (no tienen varianza mínima), y los intervalos de confianza y valores p pueden ser incorrectos. GLS ajusta el modelo para corregir estos problemas, proporcionando estimaciones más precisas y confiables.

---

### Heterocedasticidad

La heterocedasticidad ocurre cuando la **varianza de los errores cambia** según el valor de una o más variables independientes. Esto es común en datos económicos, financieros o sociales, donde por ejemplo:

- Individuos con mayores ingresos presentan más variabilidad en el gasto,
- Empresas más grandes tienen mayor variabilidad en utilidades.

En estos casos:

\begin{equation}
\text{Var}(\varepsilon_i) = \sigma_i^2 \neq \sigma^2 \quad \text{(no constante)}
\end{equation}

El modelo OLS asume erróneamente que todos los errores tienen la misma varianza. GLS corrige esto ponderando cada observación según su varianza estimada, utilizando una matriz diagonal $\boldsymbol{\Omega}$ que captura la heterocedasticidad.

---

### Correlación serial: proceso AR(1)

En datos de **series de tiempo**, es común que los errores estén **correlacionados en el tiempo**, especialmente si las observaciones se registran con alta frecuencia (horaria, diaria, etc.).

Una forma común de autocorrelación es el proceso autoregresivo de primer orden (AR(1)):

\begin{equation}
\varepsilon_t = \rho \varepsilon_{t-1} + u_t, \quad u_t \sim \mathcal{N}(0, \sigma^2)
\end{equation}

donde $\rho$ es llamado el coeficiente de autocorrelación. Esta ecuación describe un **proceso autoregresivo de primer orden**, o **AR(1)**, que modela la **dependencia temporal** entre los errores de un modelo.

#### Interpretación de los términos

- $\varepsilon_t$: error o residuo del modelo en el tiempo \( t \)
- $\varepsilon_{t-1}$: error en el tiempo anterior (\( t-1 \))
- $\rho$: coeficiente de autocorrelación (entre \(-1\) y \(1\))
- $u_t$: ruido blanco, una variable aleatoria no correlacionada e idénticamente distribuida con media cero y varianza constante
- $u_t \sim \mathcal{N}(0, \sigma^2)$: se asume que el ruido sigue una distribución normal con media cero y varianza $\sigma^2$

En este contexto, $\sigma^2$: mide la **intensidad o magnitud de la variabilidad aleatoria** en los errores que **no puede explicarse por la autocorrelación** $\rho$. Es la desviación estándar del término de innovación. Una $\sigma$ mayor implica que los errores son más variables de un periodo a otro, incluso después de considerar la dependencia con $\varepsilon_{t-1}$.


El modelo GLS supone que el valor del error actual está relacionado **linealmente** con el valor del error anterior, más una perturbación aleatoria. Es decir, **los errores no son independientes en el tiempo**, lo cual viola un supuesto clave de los modelos OLS clásicos.

- Si $\rho > 0$: los errores tienden a moverse en la misma dirección.
- Si $\rho < 0$: los errores tienden a alternar en dirección.
- Si $\rho = 0$: los errores son independientes (modelo clásico OLS).

---

En este caso, las observaciones no son independientes, y la matriz de covarianza de los errores tiene la forma de una matriz **no diagonal** con elementos que decrecen exponencialmente con la distancia temporal:

\begin{equation}
\boldsymbol{\Omega} =
\sigma^2 \begin{bmatrix}
1      & \rho   & \rho^2 & \dots  \\
\rho   & 1      & \rho   & \dots  \\
\rho^2 & \rho   & 1      & \dots  \\
\vdots & \vdots & \vdots & \ddots
\end{bmatrix}
\end{equation}

Este tipo de estructura puede ser incorporada en GLS para obtener estimaciones robustas ante autocorrelación serial.

## Importación de librerías

In [58]:
import statsmodels.api as sm

from scipy.linalg import toeplitz
# Importa la función toeplitz desde scipy.linalg. Linear algebra
# Esta función genera una matriz de Toeplitz, una matriz constante por diagonales.
# Es útil para construir matrices de covarianza con estructura autoregresiva (como AR(1)),

## Importación de datos

### El conjunto de datos Longley

El conjunto de datos **Longley** es un clásico en la historia del análisis econométrico y del modelado por regresión. Fue compilado por **J.W. Longley** en 1967 y ha sido utilizado ampliamente como ejemplo de regresión múltiple con variables altamente correlacionadas, lo que genera un entorno desafiante para la estimación de modelos lineales.

**Referencia**
Longley, J.W. (1967) "An Appraisal of Least Squares Programs for the
    Electronic Comptuer from the Point of View of the User."  Journal of
    the American Statistical Association.  62.319, 819-41.

---

### Serie temporal económica (1947–1962)

Este dataset contiene observaciones anuales de **1947 a 1962** sobre variables macroeconómicas de Estados Unidos, lo que lo convierte en una **serie temporal multivariada**. Aunque tiene pocas observaciones (16 en total), presenta características interesantes para el estudio de:

- **Multicolinealidad** severa entre variables independientes,
- **Modelado de series temporales**,
- **Diagnóstico de modelos lineales**,
- **Evaluación de predicción** en datos pequeños.

---

### Variables

| Variable | Descripción |
|----------|-------------|
| `GNP`    | Producto nacional bruto real (en miles de millones de dólares) |
| `Unemployed` | Número de personas desempleadas (en miles) |
| `Armed.Forces` | Personal empleado en las fuerzas armadas (en miles) |
| `Population` | Población total (en miles) |
| `Year` | Año calendario |
| `Employed` | Número total de personas empleadas (variable dependiente habitual) |
| `GNP.deflator` | Índice de precios implícito del PNB (base 1954 = 100) |

En muchos análisis, la variable `Employed` se toma como variable respuesta y las demás como regresores.

In [15]:
data = sm.datasets.longley.load()

In [16]:
type(data)

statsmodels.datasets.utils.Dataset

El método `datasets.longley.load()`devuelve un objeto que tiene los siguientes atributos,

- `.endog`: variable dependiente (`y`)
- `.exog`: matriz de variables explicativas (`X`)
- `.names`: lista con los nombres de todas las variables
- `.endog_name`: nombre de la variable dependiente
- `.exog_name`: nombres de las variables independientes

In [20]:
data.data # dataframe total de pandas DataFrame

,TOTEMP,GNPDEFL,GNP,UNEMP,ARMED,POP,YEAR
0,60323.0,83.0,234289.0,2356.0,1590.0,107608.0,1947.0
1,61122.0,88.5,259426.0,2325.0,1456.0,108632.0,1948.0
2,60171.0,88.2,258054.0,3682.0,1616.0,109773.0,1949.0
3,61187.0,89.5,284599.0,3351.0,1650.0,110929.0,1950.0
4,63221.0,96.2,328975.0,2099.0,3099.0,112075.0,1951.0
5,63639.0,98.1,346999.0,1932.0,3594.0,113270.0,1952.0
6,64989.0,99.0,365385.0,1870.0,3547.0,115094.0,1953.0
7,63761.0,100.0,363112.0,3578.0,3350.0,116219.0,1954.0
8,66019.0,101.2,397469.0,2904.0,3048.0,117388.0,1955.0
9,67857.0,104.6,419180.0,2822.0,2857.0,118734.0,1956.0


In [21]:
data.endog # variable dependiente

0     60323.0
1     61122.0
2     60171.0
3     61187.0
4     63221.0
5     63639.0
6     64989.0
7     63761.0
8     66019.0
9     67857.0
10    68169.0
11    66513.0
12    68655.0
13    69564.0
14    69331.0
15    70551.0
Name: TOTEMP, dtype: float64

In [22]:
data.exog # matriz de variables independientes

,GNPDEFL,GNP,UNEMP,ARMED,POP,YEAR
0,83.0,234289.0,2356.0,1590.0,107608.0,1947.0
1,88.5,259426.0,2325.0,1456.0,108632.0,1948.0
2,88.2,258054.0,3682.0,1616.0,109773.0,1949.0
3,89.5,284599.0,3351.0,1650.0,110929.0,1950.0
4,96.2,328975.0,2099.0,3099.0,112075.0,1951.0
5,98.1,346999.0,1932.0,3594.0,113270.0,1952.0
6,99.0,365385.0,1870.0,3547.0,115094.0,1953.0
7,100.0,363112.0,3578.0,3350.0,116219.0,1954.0
8,101.2,397469.0,2904.0,3048.0,117388.0,1955.0
9,104.6,419180.0,2822.0,2857.0,118734.0,1956.0


Podemos hacerlo con dataframes de pandas, pero continuamos la línea con arrays de numpy. En este contexto, tenemos que agregar manualmente la columna de 1's para crear el **intercept**.

In [23]:
data.exog = sm.add_constant(data.exog)

In [24]:
data.exog

,const,GNPDEFL,GNP,UNEMP,ARMED,POP,YEAR
0,1.0,83.0,234289.0,2356.0,1590.0,107608.0,1947.0
1,1.0,88.5,259426.0,2325.0,1456.0,108632.0,1948.0
2,1.0,88.2,258054.0,3682.0,1616.0,109773.0,1949.0
3,1.0,89.5,284599.0,3351.0,1650.0,110929.0,1950.0
4,1.0,96.2,328975.0,2099.0,3099.0,112075.0,1951.0
5,1.0,98.1,346999.0,1932.0,3594.0,113270.0,1952.0
6,1.0,99.0,365385.0,1870.0,3547.0,115094.0,1953.0
7,1.0,100.0,363112.0,3578.0,3350.0,116219.0,1954.0
8,1.0,101.2,397469.0,2904.0,3048.0,117388.0,1955.0
9,1.0,104.6,419180.0,2822.0,2857.0,118734.0,1956.0


## Aplicación de OLS

In [30]:
model_ols = sm.OLS(data.endog, data.exog)
res_ols = model_ols.fit()
res_ols.summary()

/Users/vanotole/anaconda3/envs/tsi_ml_env/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:430: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=16 observations were given.
  return hypotest_fun_in(*args, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 TOTEMP   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                     330.3
Date:                Thu, 29 May 2025   Prob (F-statistic):           4.98e-10
Time:                        14:00:21   Log-Likelihood:                -109.62
No. Observations:                  16   AIC:                             233.2
Df Residuals:                       9   BIC:                             238.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.482e+06    8.9e+05     -3.911      0.004    -5.5e+06   -1.47e+06
GNPDEFL       15.0619     84.915      0.177      0.863    -177.029     207.153
GNP           -0.0358      0.033     -1.070      0.313      -0.112       0.040
UNEMP         -2.0202      0.488     -4.136      0.003      -3.125      -0.915
ARMED         -1.0332      0.214     -4.822      0.001      -1.518      -0.549
POP           -0.0511      0.226     -0.226      0.826      -0.563       0.460
YEAR        1829.1515    455.478      4.016      0.003     798.788    2859.515
==============================================================================
Omnibus:                        0.749   Durbin-Watson:                   2.559
Prob(Omnibus):                  0.688   Jarque-Bera (JB):                0.684
Skew:                           0.420   Prob(JB):                        0.710
Kurtosis:                       2.434   Cond. No.                     4.86e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.86e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

El atributo `.resid` de los resultados del modelo guarda la información de los residuos para cada una de las observaciones. Es decir, que las variaciones al cuadrado entre el dato real y el dato predicho por el modelo para la variable dependiente. 

\begin{equation}
\sum_{i=1}^n (y_i - \hat{y}_i)^2
\end{equation}

In [29]:
ols_resid = res_ols.resid
ols_resid

0     267.340030
1     -94.013942
2      46.287168
3    -410.114622
4     309.714591
5    -249.311215
6    -164.048956
7     -13.180357
8      14.304773
9     455.394095
10    -17.268927
11    -39.055043
12   -155.549974
13    -85.671308
14    341.931514
15   -206.757825
dtype: float64

Ahora aplicamos un modelo de regresión lineal a estos residuos, como en la ecuación,

\begin{equation}
\varepsilon_t = \rho \varepsilon_{t-1} + u_t, \quad u_t \sim \mathcal{N}(0, \sigma^2)
\end{equation}

In [45]:
model_resid_ols = sm.OLS(ols_resid[1:], sm.add_constant(ols_resid[:-1].to_numpy())) 
res_resid_ols = model_resid_ols.fit()
res_resid_ols.summary()

/Users/vanotole/anaconda3/envs/tsi_ml_env/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:430: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=15 observations were given.
  return hypotest_fun_in(*args, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.137
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     2.071
Date:                Thu, 29 May 2025   Prob (F-statistic):              0.174
Time:                        14:25:37   Log-Likelihood:                -101.43
No. Observations:                  15   AIC:                             206.9
Df Residuals:                      13   BIC:                             208.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -12.8132     58.094     -0.221      0.829    -138.317     112.690
x1            -0.3634      0.253     -1.439      0.174      -0.909       0.182
==============================================================================
Omnibus:                        1.696   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.428   Jarque-Bera (JB):                0.687
Skew:                           0.523   Prob(JB):                        0.709
Kurtosis:                       3.081   Cond. No.                         230.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

De este summary, lo que nos interesa son los valores $p$ y $t$ que se utilizan en pruebas de hipótesis.

In [48]:
print('tvalues')
print(res_resid_ols.tvalues)

tvalues
const   -0.220561
x1      -1.439023
dtype: float64


In [50]:
print('pvalues')
print(res_resid_ols.pvalues)

pvalues
const    0.828860
x1       0.173784
dtype: float64


Ninguno de los p-valores se encuentra por debajo de un nivel de significancia considerable (por ejemplo, 0.05), por lo que no se puede rechazar la hipótesis nula y no hay evidencia estadística que nos diga que estos residuos siguen un proceso AR(1).

El valor de $\rho$ para este proceso está ligado a la pendiente de esta regresión.

In [54]:
rho = res_resid_ols.params[1] # únicamente para la variable x1 nos interesa
print('rho: ', rho)

rho:  -0.3634294908742479


/var/folders/ls/3cpc9bjx04xfmf_tzhgzfgrm0000gn/T/ipykernel_67936/3342983634.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rho = res_resid_ols.params[1] # únicamente para la variable x1 nos interesa


Esto nos dice que estos puntos cercanos sí tienen una correlación considerable.

In [63]:
order = toeplitz(range(len(ols_resid))) # creamos una matriz de de 16+16 para ir llenando con la covarianza de nuestros datos
order

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
       [ 1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
       [ 2,  1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13],
       [ 3,  2,  1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 4,  3,  2,  1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
       [ 5,  4,  3,  2,  1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       [ 6,  5,  4,  3,  2,  1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 7,  6,  5,  4,  3,  2,  1,  0,  1,  2,  3,  4,  5,  6,  7,  8],
       [ 8,  7,  6,  5,  4,  3,  2,  1,  0,  1,  2,  3,  4,  5,  6,  7],
       [ 9,  8,  7,  6,  5,  4,  3,  2,  1,  0,  1,  2,  3,  4,  5,  6],
       [10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0,  1,  2,  3,  4,  5],
       [11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0,  1,  2,  3,  4],
       [12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0,  1,  2,  3],
       [13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,

In [67]:
## lenamos las entradas con el valor de rho que obtuvimos elevadas a la potencia correspondiente
sigma = rho**order

El modelo de minimos cuadrados generalizado utiliza esta matriz $\boldsymbol{\Omega}$.

## Aplicación de GLS

In [68]:
model_gls = sm.GLS(data.endog, data.exog, sigma=sigma)
res_gls = model_gls.fit()
res_gls.summary()

/Users/vanotole/anaconda3/envs/tsi_ml_env/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:430: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=16 observations were given.
  return hypotest_fun_in(*args, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            GLS Regression Results                            
==============================================================================
Dep. Variable:                 TOTEMP   R-squared:                       0.998
Model:                            GLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     724.0
Date:                Thu, 29 May 2025   Prob (F-statistic):           1.48e-11
Time:                        15:26:38   Log-Likelihood:                -107.50
No. Observations:                  16   AIC:                             229.0
Df Residuals:                       9   BIC:                             234.4
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.798e+06   6.71e+05     -5.663      0.000   -5.32e+06   -2.28e+06
GNPDEFL      -12.7656     69.431     -0.184      0.858    -169.829     144.298
GNP           -0.0380      0.026     -1.448      0.182      -0.097       0.021
UNEMP         -2.1869      0.382     -5.719      0.000      -3.052      -1.322
ARMED         -1.1518      0.165     -6.970      0.000      -1.526      -0.778
POP           -0.0681      0.176     -0.386      0.709      -0.467       0.331
YEAR        1993.9529    342.635      5.819      0.000    1218.860    2769.046
==============================================================================
Omnibus:                        0.117   Durbin-Watson:                   2.611
Prob(Omnibus):                  0.943   Jarque-Bera (JB):                0.340
Skew:                          -0.036   Prob(JB):                        0.844
Kurtosis:                       2.290   Cond. No.                     5.61e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.61e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [69]:
model_glsar = sm.GLSAR(data.endog, data.exog, 1)
res_glsar = model_glsar.iterative_fit()
res_glsar.summary()

/Users/vanotole/anaconda3/envs/tsi_ml_env/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:430: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=15 observations were given.
  return hypotest_fun_in(*args, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           GLSAR Regression Results                           
==============================================================================
Dep. Variable:                 TOTEMP   R-squared:                       0.998
Model:                          GLSAR   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     801.0
Date:                Thu, 29 May 2025   Prob (F-statistic):           1.14e-10
Time:                        15:28:38   Log-Likelihood:                -100.05
No. Observations:                  15   AIC:                             214.1
Df Residuals:                       8   BIC:                             219.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.817e+06   6.28e+05     -6.078      0.000   -5.27e+06   -2.37e+06
GNPDEFL      -10.6944     66.994     -0.160      0.877    -165.182     143.793
GNP           -0.0374      0.025     -1.513      0.169      -0.094       0.020
UNEMP         -2.1921      0.361     -6.070      0.000      -3.025      -1.359
ARMED         -1.1620      0.157     -7.410      0.000      -1.524      -0.800
POP           -0.0845      0.167     -0.507      0.626      -0.469       0.300
YEAR        2004.7354    320.812      6.249      0.000    1264.942    2744.529
==============================================================================
Omnibus:                        0.265   Durbin-Watson:                   2.607
Prob(Omnibus):                  0.876   Jarque-Bera (JB):                0.433
Skew:                          -0.049   Prob(JB):                        0.805
Kurtosis:                       2.173   Cond. No.                     5.70e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.7e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Mientras que en el método `sm.GLS` uno tiene que especificar la matriz $\boldsymbol{\Omega}$, en el método `sm.GLSAR`esta estructura se estima de manera iterativa. Por tanto, en este último método se asuma una estructura AR(p) de cierto orden p en los errores, donde se quiere estimar $\rho$ automáticamente.

### Comparación de resultados

In [74]:
print('##GLS params##')
print(res_gls.params)
print('\n ##GLSAR params##')
print(res_glsar.params)
print('\n ##GLS bse##')
print(res_gls.bse)
print('\n ##GLSAR bse##')
print(res_glsar.bse)

##GLS params##
const     -3.797855e+06
GNPDEFL   -1.276565e+01
GNP       -3.800132e-02
UNEMP     -2.186949e+00
ARMED     -1.151776e+00
POP       -6.805356e-02
YEAR       1.993953e+03
dtype: float64

 ##GLSAR params##
const     -3.817410e+06
GNPDEFL   -1.069440e+01
GNP       -3.738581e-02
UNEMP     -2.192146e+00
ARMED     -1.161983e+00
POP       -8.450639e-02
YEAR       2.004735e+03
dtype: float64

 ##GLS bse##
const      670688.699310
GNPDEFL        69.430807
GNP             0.026248
UNEMP           0.382393
ARMED           0.165253
POP             0.176428
YEAR          342.634628
dtype: float64

 ##GLSAR bse##
const      628074.813842
GNPDEFL        66.993728
GNP             0.024704
UNEMP           0.361173
ARMED           0.156814
POP             0.166611
YEAR          320.811964
dtype: float64


Observamos que los coeficientes son similares entre ambos métodos.